# Assignment 1 Sistem Temu Kembali Informasi
<b> Lintang Birda Saputra (22/505034/PA/21737)</b> 
<br>
<b> Kelas KOM </b>
<br>
<b> Materi Inverted Index dan Boolean Query </b>

In [487]:
import pandas as pd
import nltk
import os
import sys
import pickle as pkl
import array
import contextlib

nltk.download('popular')


data = pd.read_csv("./News.csv")


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/lintangbs/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/lintangbs/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/lintangbs/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/lintangbs/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/lintangbs/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/lintangbs/nltk_data...
[nltk_data]    |   Package movie_review

Inverted Index

1. Explore the dataset that contains news from several authors, provide the statistical details of this document collections (how many docs, words/terms, sentences). Download the dataset from this link https://drive.google.com/file/d/13UUj240WKjosQnLfnveMcOutd0OYxBDF/view?usp=sharing  and use text under the "content" column.

2. Create inverted index based on the document collections (term based)

3. Create simple Boolean Retrieval, for merging (AND), union (OR) and negation (NOT). Check the performance of your Boolean Retrieval, by

trying at least 5 queries for each of the logical operator.

4. Record the time needed for creating inverted index and for retrieving queries.

5. Apply query optimization (if necessary) and then compare the again the time needed for retrieving queries.





Things to be submitted:

1. A 2 pages of report, explaining the analysis of the results

2. Link to the code (I prefer if you use Google Colab and provide the accessible link)

3. To write the code, please use Python.

In [488]:
data.head()

,id,id_author,title,portal,time,author,editor,content,source
0,0,1,Infografis Pekerja Asing Dilarang Masuk Wilaya...,Liputan6.com,"24 Jul 2021, 09:02 WIB",Abdillah,Abdillah,Pemerintah melalui Menteri Hukum dan Hak Asasi...,https://www.liputan6.com/news/read/4614451/inf...
1,1,1,Infografis Jadwal Bulu Tangkis Indonesia di Ol...,Liputan6.com,"23 Jul 2021, 23:23 WIB",Abdillah,Abdillah,Bulu Tangkis menjadi andalan Indonesia berburu...,https://www.liputan6.com/bola/read/4614427/inf...
2,2,1,"Infografis Jangan Bebal, Kamu Tidak Kebal Covi...",Liputan6.com,"23 Jul 2021, 10:40 WIB",Abdillah,Abdillah,Covid-19 tidak mengenal usia dan status. Siapa...,https://www.liputan6.com/news/read/4613233/inf...
3,3,1,Infografis Awas Perokok Lebih Rentan Tertular ...,Liputan6.com,"22 Jul 2021, 10:35 WIB",Abdillah,Abdillah,Kebiasaan merokok berisiko menimbulkan sejumla...,https://www.liputan6.com/news/read/4612324/inf...
4,4,1,Infografis Perbedaan Aturan PPKM Level 3 dan 4,Liputan6.com,"22 Jul 2021, 09:01 WIB",Abdillah,Abdillah,Pemberlakuan Pembatasan Kegiatan Masyarakat at...,https://www.liputan6.com/news/read/4612511/inf...


### Statistical Details of Document Collections

In [489]:
data = data.dropna(subset=['content'])
lenDocs  = len(data['content'])
print("jumlah dokumen: ",lenDocs)

totalSentences = 0
totalWords = 0
doc_total_sentences = []
doc_total_words = []
for doc in data['content']:
  totalSentences += len(nltk.sent_tokenize(doc))
  doc_total_sentences.append(len(nltk.sent_tokenize(doc)))
  totalWords += len(nltk.word_tokenize(doc))
  doc_total_words.append(len(nltk.word_tokenize(doc)))

print("total kalimat di seluruh dokumen: ", totalSentences)
print("total kata di seluruh dokumen: ", totalWords)
data= data.assign(total_sentences = doc_total_sentences)
data = data.assign(total_words = doc_total_words)

data.head()

jumlah dokumen:  14334
total kalimat di seluruh dokumen:  352491
total kata di seluruh dokumen:  6510365


,id,id_author,title,portal,time,author,editor,content,source,total_sentences,total_words
0,0,1,Infografis Pekerja Asing Dilarang Masuk Wilaya...,Liputan6.com,"24 Jul 2021, 09:02 WIB",Abdillah,Abdillah,Pemerintah melalui Menteri Hukum dan Hak Asasi...,https://www.liputan6.com/news/read/4614451/inf...,8,115
1,1,1,Infografis Jadwal Bulu Tangkis Indonesia di Ol...,Liputan6.com,"23 Jul 2021, 23:23 WIB",Abdillah,Abdillah,Bulu Tangkis menjadi andalan Indonesia berburu...,https://www.liputan6.com/bola/read/4614427/inf...,8,107
2,2,1,"Infografis Jangan Bebal, Kamu Tidak Kebal Covi...",Liputan6.com,"23 Jul 2021, 10:40 WIB",Abdillah,Abdillah,Covid-19 tidak mengenal usia dan status. Siapa...,https://www.liputan6.com/news/read/4613233/inf...,12,124
3,3,1,Infografis Awas Perokok Lebih Rentan Tertular ...,Liputan6.com,"22 Jul 2021, 10:35 WIB",Abdillah,Abdillah,Kebiasaan merokok berisiko menimbulkan sejumla...,https://www.liputan6.com/news/read/4612324/inf...,10,114
4,4,1,Infografis Perbedaan Aturan PPKM Level 3 dan 4,Liputan6.com,"22 Jul 2021, 09:01 WIB",Abdillah,Abdillah,Pemberlakuan Pembatasan Kegiatan Masyarakat at...,https://www.liputan6.com/news/read/4612511/inf...,9,114


In [490]:
class IdMap:
    """maping dari string ke id dan sebaliknya """
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []

    def __len__(self):
        return len(self.id_to_str)

    def _get_str(self, i):
        return self.id_to_str[i]

    def _get_id(self, s):
        if s not in self.str_to_id:
            self.str_to_id[s] = len(self.id_to_str)
            self.id_to_str.append(s)
        return self.str_to_id.get(s)

    def __getitem__(self, key):
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

In [491]:
class Postings:
  def encode(posting_lists):
    """encode posting lists ke stream of bytes agar bisa disimpan di file
    
    Params
    ------ 
    posting_lists: List[int]
        list of docIDs

    Returns
    ------
    bytes
        bytearray berisi integer docIds di dalam posting_list
    """
    return array.array('L', posting_lists).tobytes()

  def decode(encoded_postings_list):
    """decode posting lists dari stream of bytes
    

    Params
    ------
    encoded_postings_list: bytes
        bytearray berisi posting lists hasil dari encoding

    Returns
    ------
    List[int]
        posting_lists yang sudah didecode/list of docIDs
    """
    decoded_postings_list = array.array('L')
    decoded_postings_list.frombytes(encoded_postings_list)
    return decoded_postings_list.tolist()

In [492]:
class InvertedIndex:
    """
    Class ini menimplemntasikan method untuk read dan write inverted index ke disk

    Attr
    -------
    postings_dict = dictionary(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        memetakan termID ke metadata dari posting lists tersebut yang digunakan 
        untuk reading dan writing postings ke disk. start_position_in_index_file adalah posisi awal
        dari posting lists di dalam index file (digunakan saat file.seek(start_position_in_index_file)).
        number_of_postings_in_list adalah jumlah docIDs didalam postings.
        length_in_bytes_of_postings_list adalah ukuran postings di dalam index file (digunakan untuk 
        read posting dari index file -> file.read(length_in_bytes_of_postings_list) dari posisi start_position_in_index_file
        -> file.seek(start_position_in_index_file)).
        
        
    terms: List[int]
        list of termID
    """
    def __init__(self, index_name, postings_encoding=None, directory=''):
      """
      Params
      ------
      index_name: string
          nama index file
      postings_encoding: class
          class untuk encoding/decoding
      directory: 
          directory untuk menyimpan index files
          
      """
      self.index_file_path = os.path.join(directory, index_name+'.index')
      self.metadata_file_path = os.path.join(directory, index_name+'.dict')

      if postings_encoding is None:
          self.postings_encoding = Postings
      else:
          self.postings_encoding = postings_encoding
      self.directory = directory

      self.postings_dict = {}
      self.terms = []


    def __enter__(self):
        """Membuka file inverted index dan file metadata , serta load metadata  postings_dict, terms"""
        self.index_file = open(self.index_file_path, 'rb+')

        # load posting list dan terms dari file metadata
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()

        return self

    def __exit__(self, exception_type, exception_value, traceback):
        """"close file inverted index  dan save file inverted index"""

        self.index_file.close()

        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

### Inverted Index Creation

In [493]:
import numpy as np
import time 

class BSBIIndex:
  """Blocked sort-based Indexing Class
  1. mensegmentasi seluruh dokumen di dalam csv menjadi 11 block/bagian.
  2. sorting pasangan termID-pairID setiap block berdasarkan termID lalu sort berdasarkan pairID dan buat posting lists .
  3. menyimpan sorted posting lists ke index file di disk.
  4. merge semua block inverted index yang ada di intermediate_indices dan write ke disk (hasil akhir inverted index).

  Attrs
  ------
  term_id_map: IdMap
      untuk mapping term dari string ke termID
  doc_id_map: IdMap
    untuk mapping doc dari title string ke docID
  file_path: str
      path ke file data input csv yang akan di index
  output_dir: str
      path directory output untuk menyimpan setiap block inverted index dan merged index
  index_name: str
      nama merged index (file index berisi semua posting lists dari data input)
  postings_encoding: 
      encoding untuk menyimpan postings list ke disk (default nya UncompressedPostings).
  
    
  """
  def __init__(self, file_path, output_dir, index_name = "BSBI_lintang",
                 postings_encoding = None):
    self.term_id_map = IdMap()
    self.doc_id_map = IdMap()
    self.file_path = file_path
    self.output_dir = output_dir
    self.index_name = index_name
    self.postings_encoding = postings_encoding
    self.docLen = lenDocs
    self.intermediate_indices = []


  def save(self):
      """menyimpan doc_id_map dan term_id_map ke output directory"""

      with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
          pkl.dump(self.term_id_map, f)
      with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
          pkl.dump(self.doc_id_map, f)

  def load(self):
      """Load doc_id_map and term_id_map dari output directory"""

      with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
          self.term_id_map = pkl.load(f)
      with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
          self.doc_id_map = pkl.load(f)



  def index(self):
      """
        indexing semua document di file csv menggunakan Block sorted-based indexing
      """
      
      start = time.time()
      
      chunksize =  1434
      # data = pd.read_csv(self.file_path)
      file_name = self.file_path.split("/")[-1]
      # blocks = np.array_split(data, 10) # membagi data dokumen didalam csv menjadi 10 block bagian
      i = 0
      # read chunk of csv dengan setiap chunk berukuran 1434
      for block in pd.read_csv(self.file_path, chunksize=chunksize):
        print(len(block))
        # membuat inverted index untuk setiap block 
        td_pairs = self.parse_block(block)
        index_id = 'index_'+str(i) # membuat inverted index setiap block
        self.intermediate_indices.append(index_id) 
        i +=1
        with InvertedIndexWriter(index_id, directory=self.output_dir,
                                  postings_encoding=
                                  self.postings_encoding) as index:
            # membuka file index_id dan write posting list ke dalam file tersebut
            # setelah itu ke block berikutnya dan close file index_id
            self.invert_write(td_pairs, index)
            td_pairs = None
    

      self.save()
      with InvertedIndexWriter(self.index_name, directory=self.output_dir,
                                postings_encoding=
                                self.postings_encoding) as merged_index:
          # membuka file index_name dan merge semua block inverted index ke merged_index
          # write merged_index ke file index_name
          with contextlib.ExitStack() as stack:
              indices = [stack.enter_context(
                  InvertedIndexIterator(index_id,
                                        directory=self.output_dir,
                                        postings_encoding=
                                        self.postings_encoding)) # membuka file index_id (setiap block)
                
                for index_id in self.intermediate_indices] # berisi semua block inverted index  
              
              self.merge(indices, merged_index)

      end = time.time()
      print(f"total waktu yang dibutuhkan untuk indexing seluruh dokumen: {(end-start)*10**3:.03f}ms")
    

In [494]:
  # for block in blocks:
      #   # membuat inverted index untuk setiap block 
      #   td_pairs = self.parse_block(block)
      #   index_id = 'index_'+str(i) # membuat inverted index setiap block
      #   self.intermediate_indices.append(index_id) 
      #   i +=1
      #   with InvertedIndexWriter(index_id, directory=self.output_dir,
      #                             postings_encoding=
      #                             self.postings_encoding) as index:
      #       # membuka file index_id dan write posting list ke dalam file tersebut
      #       # setelah itu ke block berikutnya dan close file index_id
      #       self.invert_write(td_pairs, index)
      #       td_pairs = None

In [495]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()


stop_words = set(nltk.corpus.stopwords.words('indonesian'))
class BSBIIndex(BSBIIndex):
    def parse_block(self, block):
      """
        Parse satu block yang berisi list of documents dari csv menjadi termID-docID pairs
    
        Params
        ------
        block: pandas.Dataframe
           satu block dataframe (hasil dari read csv dengan ukuran chunksize)

        Returns
        ------
        List[tuple[Int, Int]]
            list semua termID, docID pairs
        
      """
      term_doc_pairs = []

      for ind in block.index:
        
        doc = block["content"][ind]
        if type(doc) != str:
          continue
        processed_word_in_doc = []
        doc_id = self.doc_id_map[block["title"][ind]]
        doc = stemmer.stem(doc) # stemming
        words = nltk.word_tokenize(doc) # tokenisasi
        for word in words:
          word = word.lower()  # mengubah kata ke lowercase
          if word not in stop_words: # stopword removal
            term_id = self.term_id_map[word]
            term_doc_pairs.append([term_id, doc_id])
      return term_doc_pairs



In [496]:
class InvertedIndexWriter(InvertedIndex):

  def __enter__(self):
    """ 
    membuka file index_file_path , berisi inverted index untuk block tertentu dari dataset
    """
    self.index_file = open(self.index_file_path, 'wb+')
    return self

  def append(self, term, postings_list):
    """
    append term dan posting lists ke index file (posisi di block paling akhir di file)

    posting_dict memetakan termID ke tuple, isi posting_dict:
    (start_position_in_index_file,
           number_of_postings_in_list,
           length_in_bytes_of_postings_list)
    """
    encoded_postings_list = self.postings_encoding.encode(postings_list) # bytearrray dari posting_lists
    start_position_in_index_file = self.index_file.seek(0, 2) # posisi paling akhir di file
    length_in_bytes_of_postings_list = self.index_file.write(encoded_postings_list) # write bytearray posting_lists ke file index
    self.terms.append(term) # tambahkan term ke list of terms
    self.postings_dict[term] = (start_position_in_index_file, len(postings_list),length_in_bytes_of_postings_list ) # simpan metadata dari term



In [497]:
from collections import defaultdict


class BSBIIndex(BSBIIndex):
  def invert_write(self, td_pairs, index):
    """
    Invert td_pairs menjadi posting lists .
     sorting berdasarkan termID lalu sort lagi berdasarkan docID.
    lalu simpan posting list ke file indeex melalui method append() dari InvertedIndexWriter
    
     Params
     ------
     td_pairs: List[Tuple[Int,Int]]
         list semua termID, docID pairs
     index: InvertedIndexWriter
         satu block inverted index 
    """
    term_doc_dict =  defaultdict(list)
    for t,d in td_pairs:
        term_doc_dict[t].append(d)
    for t in sorted(term_doc_dict.keys()):
      unsorted_posting_list = term_doc_dict[t]

      sorted_posting_list = sorted(unsorted_posting_list)
      index.append(t, sorted(sorted_posting_list))


In [498]:
class InvertedIndexIterator(InvertedIndex):
  """
        iterator untuk setiap block inverted index.
        read file dari disk hanya dilakukan saat process merge (ketika heapq.merge(*indices) 
            method __next__ dari iterator akan dipanggil dan read posting lists dari term ke disk).
       setiap iterasi, yang diload ke memory hanya satu posting list dari disk. tujuannya agar posting lists yang 
       di load ke memory tidak terlalu besar/tidak semua posting lists yang di 
        disk di load ke memory. 
  """
  def __enter__(self):
    super().__enter__()
    self._initialization_hook()
    return self 

  def _initialization_hook(self):
    self.curr_term_pos = 0 

  def __iter__(self):
    return self

  def __next__(self):
    """
        return  pasangan (termID, posting_lists) selanjutnya

        read satu posting lists dari term dari index file/disk

        Returns
        -----
        Int
            termID yang di read
        List[Int]
            posting lists dari termID
        
    """
    if self.curr_term_pos < len(self.terms):
        term = self.terms[self.curr_term_pos]
        self.curr_term_pos += 1
        start_position, n_postings, length_in_bytes = self.postings_dict[term]
        self.index_file.seek(start_position)
        postings_list = self.postings_encoding.decode(self.index_file.read(length_in_bytes))
        return term, postings_list
    else:
        raise StopIteration



  def delete_from_disk(self):

      self.delete_upon_exit = True

  def __exit__(self, exception_type, exception_value, traceback):
      """
        close file index
      """
      self.index_file.close() 
      
      if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
          os.remove(self.index_file_path)
          os.remove(self.metadata_file_path)
      else:
          with open(self.metadata_file_path, 'wb') as f:
              pkl.dump([self.postings_dict, self.terms], f)

In [499]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """
        merge semua blok inverted index menjadi satu index (merged_index)
        dan write merged_index ke file index (melalui method append() dari InvertedIndexWriter

        Params
        ------
        indices: List[InvertedIndexIterator]
            list of InvertedIndexIterator, setiap elemen nya adalah satu block inverted index (hasil
            dari split dokumen menjadi 11 block inverted index).
        merged_index: InvertedIndexWriter
            hasil akhir inverted index, dimana setiap block inverted index dimerge ke merged_index ini.
        """
        last_term = last_posting = None
        for curr_term, curr_postings in heapq.merge(*indices):
            # heapq.merge mereturn list of curr_term dan curr_postings yang sudah disort berdasarkan term dan postings_list
            # posting lists didapatkan dari read block of bytes ke index file yang ada didalam disk (method __next__ InvertedIndexIterator)
            # 
            if curr_term != last_term:
                if last_term:
                    last_posting = list(sorted(set(last_posting))) # sort previous posting lists berdasarkan docIDs
                    merged_index.append(last_term, last_posting) # write previous merged posting lists ke file index
                last_term = curr_term
                last_posting = curr_postings
            else:
                last_posting += curr_postings
        if last_term:
            last_posting = list(sorted(set(last_posting))) 
            merged_index.append(last_term, last_posting) # write merged posting lists ke file index

In [500]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)

    def _get_postings_list(self, term):
        """"
        mendapatkan postinglists dari term

        Params
        -----
        Int
            termID 
        """
        start_position, n_postings, length_in_bytes = self.postings_dict[term]
        self.index_file.seek(start_position)
        return self.postings_encoding.decode(self.index_file.read(length_in_bytes))


In [501]:
def sorted_intersect(list1, list2):
    """

    operasi intersection dari list1 dan list2.
    list1 dan list2 sorted by docID.

    Params
    ----
    list1: List[Int]
        posting lists ke-1
    list2: List[Int]
        posting lists ke-2

    Returns
    -----
    List[Int]
        hasil dari operasi intersection
    """
    idx1 = idx2 = 0
    intersect = []
    while idx1 < len(list1) and idx2 < len(list2):
        if list1[idx1] < list2[idx2]:
            idx1 += 1
        elif list2[idx2] < list1[idx1]:
            idx2 += 1
        else:
            intersect.append(list1[idx1])
            idx1 += 1
            idx2 += 1
    return intersect

def sorted_union(list1, list2):
    """

    operasi union dari list1 dengan list2.
    list1 dan list2 sorted by docID.

    Params
    ----
    list1: List[Int]
        posting lists ke-1
    list2: List[Int]
        posting lists ke-2

    Returns
    -----
    List[Int]
        hasil dari operasi union
    """
    idx1 = idx2 = 0
    union = []
    while idx1 < len(list1) and idx2 < len(list2):
        if list1[idx1] < list2[idx2]:
            union.append(list1[idx1])
            idx1 += 1
        elif list1[idx1] > list2[idx2]:
            union.append(list2[idx2])
            idx2 += 1
        else:
            union.append(list1[idx1])
            idx1 +=1
            idx2 += 1
    if idx1 == len(list1):
        while(idx2 < len(list2)):
            union.append(list2[idx2])
            idx2 += 1
    else:
        while(idx1 < len(list1)):
            union.append(list1[idx1])
            idx1 += 1
    return union


def shuntingYardRPN(tokens):
    """
    mebuat reverse polish notations dari list of tokens
    dengan menggunakan shunting yard algorithm
    
    Params
    ----
    tokens: List[str]
        list of token.

    Returns
    -----
    List[str]
        hasil reverse polish notation/postfix notation dari shunting yard algorithm
    """
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0 
    
    output = []
    stack = []

    for token in tokens:
        if (token == '('):
            stack.append(token)

        elif (token == ')'):
            operator = stack.pop()
            while operator != '(':
                output.append(operator)
                operator = stack.pop()

        elif (token in precedence):
            if (stack):
                operator = stack[-1]
                while (stack and precedence[token] < precedence[operator]):
                    output.append(operator)
                    stack.pop()
                    if (stack):
                        operator = stack[-1]
            
            stack.append(token)
        else:
            # term 
            output.append(token.lower())
    
    while (stack):
        output.append(stack.pop())
    return output


def not_operation(list1, indexed_docIDs):
    """
    operasi negasi untuk list1
    
    Params
    ----
    list1: List[int]
        posting lists ke-1
    indexed_docIDs: List[int]
        semua docIDs yang sudah di index oleh search engine

    Returns
    -----
    List[Int]
        hasil operasi negasi
    """
    if (not list1):
        return indexed_docIDs

    result = []
    list1Set = set(list1)
    for item in indexed_docIDs:
        if item not in list1Set:
            result.append(item)
    return result
    

In [744]:
import collections
import re

class BSBIIndex(BSBIIndex):
  def retrieve(self, query, tipe):
      """
      Query Boolean retrieval dengan operasi sesuai dengan typenya (AND, OR, NOT)
      """
      if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
          self.load()

      with InvertedIndexMapper(self.index_name, directory=self.output_dir,
                                postings_encoding=
                                self.postings_encoding) as mapper:
          result = None
         
          for term in query.split():
              term =  term.lower()
              term_id = self.term_id_map.str_to_id.get(term)
              if not term_id:
                  return []
              r = mapper[term_id]
              if result is None:
                  result = r
              else:
                  if tipe == "AND":
                      result = sorted_intersect(result, r)
                  elif tipe == "OR":
                      result = sorted_union(result, r)
      return [self.doc_id_map[r] for r in result] # r = docID, doc_id_map[r] = string dari document

    
  def query(self, query):
      """
      boolean query (support operasi AND, OR, dan Negation).
      
      Params
      ------
      query: str
          query untuk search engine

      Returns
      ------
      List[str]
          list of title documents yang relevan dengan query
      """
      start = time.time()
      if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
          self.load()

      with InvertedIndexMapper(self.index_name, directory=self.output_dir,
                                postings_encoding=
                                self.postings_encoding) as mapper:
          result = None
          
          result = self.processQuery(query, mapper)
          end = time.time()
          print(f"total waktu yang dibutuhkan untuk melakukan query (non-optimized): {(end-start)*10**3:.03f}ms")
          return [self.doc_id_map[r] for r in result]
          

  def processQuery(self,query, mapper):
        """"
        memproses query dengan cara membuat reverse polish notations dari string query, lalu
        mengevaluasi reverse polish notations sesuai dengan tipe operator nya (AND/OR/NOT).

        Params
        -------
        query: str
            string query ke search engine
        mapper: InvertedIndexMapper
            mapper untuk memetakan termID ke postingListsnya

        Returns
        ------
        List[int]
            list of docID yang relevan dengan query
        """
        
        query = query.replace('(', '( ')
        query = query.replace(')', ' )')
        query  = query.split(' ')

        indexed_docIDs = list(range(1, self.docLen +1))
                              
      
        stack = []
        rpn_queue = collections.deque(shuntingYardRPN(query))

        while rpn_queue:
            token = rpn_queue.popleft()
            if token not in ('AND', 'NOT', 'OR'):
                # term
                token = stemmer.stem(token)
                term_id = self.term_id_map.str_to_id.get(token.lower())
                posting_list = mapper[term_id]
                stack.append(posting_list)
            else:
                if (token == 'AND'):
                    right = stack.pop()
                    left = stack.pop()
                    stack.append(sorted_intersect(left, right))
                elif (token == 'OR'):
                    right = stack.pop()
                    left = stack.pop()
                    stack.append(sorted_union(left, right))
                elif (token == 'NOT'):
                    right = stack.pop()
                    stack.append(not_operation(right, indexed_docIDs))
        if len(stack) != 1:
            print("error, harusnya panjang stacknya tinggal 1")
            return None
        return stack.pop()
                
  def queryOptimization(self, query):
      start = time.time()
      if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
          self.load()

      with InvertedIndexMapper(self.index_name, directory=self.output_dir,
                                postings_encoding=
                                self.postings_encoding) as mapper:
         result = None
         query = re.findall('[a-zA-Z0-9]+', query, flags=re.IGNORECASE)
         prev = None
         pairs = []
         for token in query:
              token = stemmer.stem(token)
              term_id = self.term_id_map.str_to_id.get(token.lower())
              posting_list = mapper[term_id]
              print(token)
              print(len(posting_list))
              if token not in ('and', 'not', 'or'):
                  if prev != None:
                      pairs.append([prev, token])
                  prev = token
         processed = set()
         res = []
         for pair in pairs:
             token1 = stemmer.stem(pair[0])
             term_id1 = self.term_id_map.str_to_id.get(token1.lower())
             posting_list1 = mapper[term_id1]

             token2 = stemmer.stem(pair[1])
             term_id2 = self.term_id_map.str_to_id.get(token2.lower())
             posting_list2 = mapper[term_id2]
             
             
             if term_id1 in processed:
                other = posting_list2
                res = sorted_intersect(res, other)
             elif term_id2 in processed:
                other = posting_list1
                res = sorted_intersect(res, other)
             else:
              
                res = sorted_intersect(posting_list1, posting_list2) # list of docIDs
             processed.add(term_id1)
             processed.add(term_id2)
         return [self.doc_id_map[r] for r in res]
                
                 
      
      end = time.time()
      print(f"total waktu yang dibutuhkan untuk melakukan query (non-optimized): {(end-start)*10**3:.03f}ms")
      
    

In [745]:
BSBI_instance = BSBIIndex(file_path= "./News.csv", output_dir = 'output_dir', )
BSBI_instance.index()

1434
1434
1434
1434
1434
1434
1434
1434
1434
1434
3
total waktu yang dibutuhkan untuk indexing seluruh dokumen: 24400.308ms


### 5 Boolean Query non-optimized


In [746]:
BSBI_instance.query('Joget AND Tiktok')

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 20.197ms


['Dua Pria Ini Asik Joget TikTok Sambil Nyetir Mobil Hingga Berujung Apes Masuk Empang',
 'Beda Pelayanan kepada Tamu Lokal dan Turis Asing di Bali Jadi Viral',
 'Viral Video Dua Remaja Joget TikTok Hampir Ketabrak Motor di Klaten']

In [747]:
BSBI_instance.query('Jokowi AND Gibran')

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 21.357ms


['Putra Jokowi, Gibran Rakabuming Raka Positif Covid-19',
 'Positif Covid-19, Gibran: Saya Masih Bekerja 24 Jam untuk Warga Solo',
 'Positif Covid-19, Gibran: Istri dan Anak Semuanya Negatif',
 '5 Pengakuan Gibran Rakabuming Usai Dinyatakan Positif Covid-19',
 'Top 3 News: Gibran Rakabuming Raka Positif Covid-19',
 'Media Sosial dan Efektivitas Kritik Mahasiswa untuk Pemerintah',
 'Polisi Militer Saksikan Gibran Rakabuming Semprotkan Desinfektan ke Ajudan',
 'Wanita Cantik Minta Selfie Sama Gibran, Ternyata Anak Buah Pejabat Tinggi di RI',
 'Dibilang Lumpuh Total, Reaksi Gibran Rakabuming Santuy Banget Bikin Salut',
 'Viral Video Lawas Momen Gibran Rakabuming yang Bikin Najwa Shihab Deg-Degan',
 'Bagyo Wahyono Pesaing Gibran di Solo: Saya Bentuk Tangisan dari Wong Cilik']

In [748]:
BSBI_instance.query('Olimpiade AND Tokyo OR Kalah')

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 22.006ms


['Infografis Jadwal Bulu Tangkis Indonesia di Olimpiade Tokyo 2020',
 'Hasil Menembak Olimpiade Tokyo 2020 : Vidya Rafika Gagal Buka Peluang Raih Medali',
 'Top 3 Berita Bola : Simak Jadwal Lengkap Olimpiade Tokyo 2020',
 'Jadwal Siaran Langsung Atlet Indonesia di Olimpiade Tokyo 2020, Sabtu 24 Juli 2021',
 'Link Live Streaming Bulu Tangkis Indonesia Olimpiade Tokyo 2020 di Vidio : Misi Dulang Medali Dimulai',
 'Jadwal Olimpiade Tokyo 2020 : Tim Inggris Raya Ingin Tampil Tanpa Emosi di Laga Pembuka',
 'Waduh, Ahli Kesehatan Serukan Penundaan Olimpiade Tokyo 2020 Usai Belasan Atlet Positif Covid-19',
 'Target Buruan Segera Gabung, MU Kebingungan Jual Pemain Buangan',
 'Juventus Pertimbangkan Tukar Ronaldo dengan Striker Klub Prancis',
 'Lama Jadi Masalah Besar, Lini Serang MU Akhirnya Bisa Diperbaiki Berkat 2 Pemain Ini',
 'Jadwal Bulu Tangkis Olimpiade 2020 : Andalan Indonesia Dulang Medali',
 'Pelatih Puas Jepang Tahan Imbang Spanyol di Laga Pemanasan Olimpiade Tokyo',
 'Usain Bolt Kh

In [749]:
gb2 = BSBI_instance.query(query='Gibran AND Rakabuming AND Solo')
for res in gb2:
    print(res)

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 24.880ms
Gojek Dukung Percepatan Vaksinasi di PPKM Darurat dengan Diskon GoRide dan GoCar
Putra Jokowi, Gibran Rakabuming Raka Positif Covid-19
Positif Covid-19, Gibran: Saya Masih Bekerja 24 Jam untuk Warga Solo
Positif Covid-19, Gibran Rakabuming Sering Kunjungi Rumah Sakit hingga Tempat Isolasi Pasien
Positif Covid-19, Gibran: Istri dan Anak Semuanya Negatif
5 Pengakuan Gibran Rakabuming Usai Dinyatakan Positif Covid-19
Top 3 News: Gibran Rakabuming Raka Positif Covid-19
Media Sosial dan Efektivitas Kritik Mahasiswa untuk Pemerintah
Polisi Militer Saksikan Gibran Rakabuming Semprotkan Desinfektan ke Ajudan
Wanita Cantik Minta Selfie Sama Gibran, Ternyata Anak Buah Pejabat Tinggi di RI
Berstatus Anak Presiden, Begini Gaya Gibran Sambut Panglima TNI dan Kasad Andika
Gibran Anak Jokowi Tinggalkan Mobil Dekat Makam Dirusak, Sampai Malam Dijaga Warga
Gibran Izinkan Mal di Solo Tetap Buka, Tapi dengan Syarat


In [750]:
jk2= BSBI_instance.query(query='Jokowi AND Prabowo')
for res in jk2:
    print(res)

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 23.169ms
Agar Bahagia, Sapi Kurban Jokowi di Surabaya Dijaga 24 Jam
Eks Menteri KKP Edhy Prabowo Hadapi Vonis, KPK Harap Hakim Pertimbangkan Fakta Hukum
Ketua Satgas Covid-19: Ajak Keluarga Vaksinasi, Agar Jaga Diri dan Sehat
Ada Ajakan Unjuk Rasa, Polda Metro Jaya: Rumah Sakit dan Kuburan Sudah Penuh
Upaya Mempercepat Terbentuknya 'Herd Immunity' di Banten
Dilema Politisi Sekaligus Menteri
Survei Top Of Mind Voxpol: Ganjar Kalahkan Jokowi dan Prabowo
Koalisi 'Alumni Golkar' Bisa Jadi Ancaman Serius Bagi Dominasi PDIP di 2024
SMRC: Massa Gerindra dan PKS Paling Keras Tolak Jabatan Presiden Lebih dari 2 Periode
Tolak Usulan Relawan Jokowi, PPP Ingin Masa Jabatan Presiden Tetap 2 Periode
Usulan Jokowi 3 Periode Dinilai Karena Ada Pihak Tak Mau Kehilangan Kekuasaan
Pengamat: Ganjar-Sandi Mampu 'Beli' Kendaraan Politik Demi Maju 2024
Survei SMRC: Prabowo Unggul di Pemilih yang Tak Puas dengan Jokowi
Prabowo Ungkap Rencana P

In [751]:
jgp = BSBI_instance.query(query='Ganjar AND PDIP AND Megawati AND NOT Jokowi')
for res in jgp:
    print(res)

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 26.695ms
Survei SMRC: Elektabilitas Prabowo-Ganjar Naik, Anies di Posisi Ketiga
Wasekjen PDIP Mengaku Belum Dengar Soal Teguran Megawati ke Bambang Pacul
Utut Adianto: Saya Sarankan Puan Maharani Konsentrasi sebagai Ketua DPR
PDIP: Megawati Tidak Mengurusi Ribut-ribut Ganjar Pranowo dan Puan Maharani
Usai jadi Host Youtube, Sepak Terjang Ganjar Pranowo Dianggap Sudah Kelewatan
Survei: Bukan Prabowo, Milenial dan Gen Z Unggulkan Ganjar-Ridwan Kamil
Saran Saiful Mujani Buat Ganjar Pranowo: Masuk Timses Puan Maharani
Survei LP3ES: PDIP Teratas, Demokrat Salip Dominasi Gerindra dan Golkar
Membaca Peluang Koalisi Nasionalis-Religius Usung Capres di 2024
DPP PDIP Tegaskan Tidak Ada Keputusan Persilakan Ganjar Pranowo Cari Parpol Lain
Cerita Puan Maharani Dimarahi Megawati
Politikus PDIP: Pertemuan Mega, Puan, Hasto dan Pacul Bahas Hal Super Spesifik


In [752]:
jgp = BSBI_instance.query(query='(Jokowi AND Ganjar) AND Prabowo')
for res in jgp:
    print(res)


total waktu yang dibutuhkan untuk melakukan query (non-optimized): 27.610ms
Dilema Politisi Sekaligus Menteri
Survei Top Of Mind Voxpol: Ganjar Kalahkan Jokowi dan Prabowo
Pengamat: Ganjar-Sandi Mampu 'Beli' Kendaraan Politik Demi Maju 2024
Survei SMRC: Prabowo Unggul di Pemilih yang Tak Puas dengan Jokowi
Survei: Jokowi Memimpin Elektabilitas Capres Jika Boleh Nyapres di 2024
Survei: 80,9 Persen Milenial dan Gen Z Puas Terhadap Kepemimpinan Jokowi
Ganjar Tanggapi Wacana Duet dengan Sandiaga di Pemilu 2024
Elektabilitas AHY dan Partai Naik, Demokrat Jadikan Survei sebagai Vitamin
Ganjar Vs Puan, Megawati di Antara Pilihan Rasional atau Emosional
Jokowi Mania Sebut Ganjar-Sandi Duet Maut di Pilpres 2024
Koalisi PDIP-Gerindra: Pilih Prabowo-Puan atau Ganjar-Sandi?


In [753]:
jgp = BSBI_instance.query(query='(Jokowi AND Ganjar) AND (Pemilu OR Gibran)')
for res in jgp:
    print(res)


total waktu yang dibutuhkan untuk melakukan query (non-optimized): 21.490ms
Dilema Politisi Sekaligus Menteri
Survei: 80,9 Persen Milenial dan Gen Z Puas Terhadap Kepemimpinan Jokowi
Elektabilitas AHY dan Partai Naik, Demokrat Jadikan Survei sebagai Vitamin
Panas Puan Vs Ganjar, PDIP Ingatkan Jangan Beri Peluang Upaya Pecah Belah Partai
Ganjar Vs Puan, Megawati di Antara Pilihan Rasional atau Emosional


In [754]:
ss = BSBI_instance.query(query='(Jokowi AND Ganjar) AND NOT (Pemilu OR Gibran)')
for res in ss:
    print(res)


total waktu yang dibutuhkan untuk melakukan query (non-optimized): 23.475ms
PGN Optimalkan Peran Gas Bumi dalam Akselerasi Kawasan Ekonomi Baru
Survei Top Of Mind Voxpol: Ganjar Kalahkan Jokowi dan Prabowo
Pengamat: Ganjar-Sandi Mampu 'Beli' Kendaraan Politik Demi Maju 2024
Survei SMRC: Prabowo Unggul di Pemilih yang Tak Puas dengan Jokowi
Fenomena Munculnya Relawan, Demi Menjaga Nama Ganjar Pranowo Tetap Eksis
Ketua DPP PDIP: Pernyataan Pacul Soal Puan Sikap Pribadi
Survei: Jokowi Memimpin Elektabilitas Capres Jika Boleh Nyapres di 2024
Ganjar Tanggapi Wacana Duet dengan Sandiaga di Pemilu 2024
Jokowi Mania Sebut Ganjar-Sandi Duet Maut di Pilpres 2024
Koalisi PDIP-Gerindra: Pilih Prabowo-Puan atau Ganjar-Sandi?


In [755]:
ss = BSBI_instance.query(query='Militer AND Kampus AND NOT Harimau')
for res in ss:
    print(res)

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 24.079ms
Ikat Kepala 'Reformasi Damai' & Memoar Aksi Mei 1998 di Purwokerto
Seorang Mahasiswi di Aceh Mual dan Muntah-Muntah Usai Disuntik Vaksin Covid-19
Kampus Ruang Diskusi, Pendidikan Militer Belum Urgensi
Sinyal Program Militer Masuk Kampus


In [756]:
ss = BSBI_instance.query(query='Pekerja AND Asing AND Indonesia')
for res in ss:
    print(res)

total waktu yang dibutuhkan untuk melakukan query (non-optimized): 21.720ms
Infografis Pekerja Asing Dilarang Masuk Wilayah Indonesia
Laju IHSG Galau, Investor Asing Kejar Saham ANTM hingga INDF
8 Jenis-Jenis Tabungan di Indonesia dan Manfaatnya
Giro adalah Produk Simpanan, Ketahui Jenis dan Manfaatnya
Top 3: Diskon Tarif Listrik Diperpanjang hingga Meninggalnya Christianto Wibisono
Vaksinasi Berbayar Batal, Istana: Semua Vaksin Covid-19 Gratis
Naik 19 Kali Lipat, Investor Saham Syariah Tembus 93.870 di 2021
Erick Thohir Uji Coba Integrasi Dokumen Syarat Perjalanan Udara di Aplikasi PeduliLindungi
Erick Thohir Usul BNI Dapat PMN Rp 7 Triliun di 2022
Erick Thohir Targetkan Setoran Dividen BUMN Naik Jadi Rp 35 Triliun di 2021
Komisi III DPR Minta Larangan TKA Masuk Indonesia Tak Pandang Bulu
Update Covid-19: Kematian Tembus 1.205 Jadi 71.397 pada Jumat 16 Juli 2021
Bertambah 49.509, Kasus Positif Covid-19 Per Kamis 22 Juli 2021 Tembus 3.033.339
Update Covid-19 Rabu 21 Juli 2021: Positif 

In [758]:
ss2 = BSBI_instance.queryOptimization(query='Pekerja AND Asing AND Indonesia')
for res in ss2:
    print(res)


kerja
4390
and
459
asing
585
and
459
indonesia
4830
Infografis Pekerja Asing Dilarang Masuk Wilayah Indonesia
Laju IHSG Galau, Investor Asing Kejar Saham ANTM hingga INDF
8 Jenis-Jenis Tabungan di Indonesia dan Manfaatnya
Giro adalah Produk Simpanan, Ketahui Jenis dan Manfaatnya
Top 3: Diskon Tarif Listrik Diperpanjang hingga Meninggalnya Christianto Wibisono
Vaksinasi Berbayar Batal, Istana: Semua Vaksin Covid-19 Gratis
Naik 19 Kali Lipat, Investor Saham Syariah Tembus 93.870 di 2021
Erick Thohir Uji Coba Integrasi Dokumen Syarat Perjalanan Udara di Aplikasi PeduliLindungi
Erick Thohir Usul BNI Dapat PMN Rp 7 Triliun di 2022
Erick Thohir Targetkan Setoran Dividen BUMN Naik Jadi Rp 35 Triliun di 2021
Komisi III DPR Minta Larangan TKA Masuk Indonesia Tak Pandang Bulu
Update Covid-19: Kematian Tembus 1.205 Jadi 71.397 pada Jumat 16 Juli 2021
Bertambah 49.509, Kasus Positif Covid-19 Per Kamis 22 Juli 2021 Tembus 3.033.339
Update Covid-19 Rabu 21 Juli 2021: Positif 2.983.830, Sembuh 2.356.

In [759]:
print(ss == ss2)

True


In [761]:
ll = ['a', 'AND', '(', 'b', 'OR', 'C', 'OR', 'D', ')', 'AND', 'E']
ss = shuntingYardRPN(ll)
print(ss)

['a', 'b', 'c', 'd', 'OR', 'OR', 'e', 'AND', 'AND']


In [762]:
ll = ['a', 'AND', '(', 'b', 'OR', 'C', 'OR', 'D', 'OR', 'L', ')', 'AND', 'E']
ss = shuntingYardRPN(ll)
print(ss)

['a', 'b', 'c', 'd', 'l', 'OR', 'OR', 'OR', 'e', 'AND', 'AND']


In [763]:
ll = [ '(', 'b', 'OR', 'C', ')', 'AND', '(', 'L', 'OR' , 'S', ')', 'AND', '(', 'Q', 'OR', 'M', ')' ]
ss = shuntingYardRPN(ll)
print(ss)

['b', 'c', 'OR', 'l', 's', 'OR', 'q', 'm', 'OR', 'AND', 'AND']
